In [1]:
import mysql.connector
from credentials import username, password

In [11]:
con = mysql.connector.connect(host="127.0.0.1", port=3307,
                              database="tolstoyphotos",
                              user=username, password=password)
cur = con.cursor(dictionary=True)

## Запросы с `SELECT` и `JOIN`

Время спорта! Толстой играл в теннис, а ещё упоминал его в тексте «Анны Карениной». Найдётся ли хотя бы одна фотография с корта?

In [3]:
cur.execute("""
SELECT * FROM PhotoDescriptions AS PD
    JOIN PhotoRubrics AS PR ON PR.idPhoto=PD.idPhoto
    JOIN Rubrics AS R ON R.idRubric=PR.idRubric
WHERE rubricName LIKE '%теннис%'
""")
cur.fetchall()

[{'idPhoto': 97194,
  'idAuthor': 221,
  'idLocation': 319,
  'year': 1896,
  'photoDescription': 'Слева на право: Л.Н. Толстой, М.Л. Оболенская,  и А.Л. Толстая, Н.Л. Оболенский.',
  'idRubric': 166,
  'rubricName': 'теннис'},
 {'idPhoto': 727,
  'idAuthor': 0,
  'idLocation': 319,
  'year': 1896,
  'photoDescription': '-1',
  'idRubric': 166,
  'rubricName': 'теннис'},
 {'idPhoto': 135648,
  'idAuthor': 221,
  'idLocation': 319,
  'year': 1897,
  'photoDescription': 'Слева направо: Л. Н. Толстой, Н. Л. Оболенский, Александра и Мария Толстые, С. А. Толстая, А. Б. Гольденвейзер, Т. Л. Толстая.',
  'idRubric': 166,
  'rubricName': 'теннис'},
 {'idPhoto': 97194,
  'idAuthor': 221,
  'idLocation': 319,
  'year': 1896,
  'photoDescription': 'Слева на право: Л.Н. Толстой, М.Л. Оболенская,  и А.Л. Толстая, Н.Л. Оболенский.',
  'idRubric': 166,
  'rubricName': 'теннис'},
 {'idPhoto': 135648,
  'idAuthor': 221,
  'idLocation': 319,
  'year': 1897,
  'photoDescription': 'Слева направо: Л. Н. То

Вот одна из этих фотографий — id 92715:
![](./data/fullsize/00092715.jpg)

Более сложный `SELECT`, с группировкой, подсчётом и сортировкой результата: смотрим, кто из авторов был наиболее разнообразен. Разнообразие считаем по количеству рубрик, в которых есть хотя бы одна фотография этого автора.

In [4]:
cur.execute(
"""
SELECT 
    A.authorName,
    COUNT(R.idRubric) AS rubricsCount
FROM PhotoDescriptions AS PD
    JOIN PhotoRubrics AS PR ON PR.idPhoto=PD.idPhoto
    JOIN Rubrics AS R ON R.idRubric=PR.idRubric
    JOIN Authors AS A ON A.idAuthor=PD.idAuthor
GROUP BY A.idAuthor
ORDER BY rubricsCount DESC
LIMIT 10;
""")
cur.fetchall()

[{'authorName': '-1', 'rubricsCount': 5187},
 {'authorName': 'Чертков В.Г.', 'rubricsCount': 833},
 {'authorName': 'Толстая С.А. (Берс)', 'rubricsCount': 329},
 {'authorName': 'Гринберг А.Д., Живаго В.Р.', 'rubricsCount': 206},
 {'authorName': 'Кулаков П.Е.', 'rubricsCount': 203},
 {'authorName': 'Булла К.К.', 'rubricsCount': 170},
 {'authorName': 'Сергеенко П.А.', 'rubricsCount': 140},
 {'authorName': 'Толстая С.А.', 'rubricsCount': 137},
 {'authorName': 'Деньер А.И.', 'rubricsCount': 102},
 {'authorName': 'Протасевич Ф.Т.', 'rubricsCount': 69}]

## Вложенный `SELECT`

Зачем отдельная таблица с локациями и таблица с локациями как из музея? Вот пример: мириады вариантов написания Ясной Поляны.

In [12]:
cur.execute("""
SELECT 
    DISTINCT(MA.originalLocation)
FROM PhotoDescriptions AS PD
    JOIN Locations AS L ON PD.idLocation=L.idLocation
    JOIN MuseumAnnotations AS MA ON PD.idPhoto=MA.idPhoto
WHERE L.idLocation IN (SELECT idLocation FROM Locations WHERE locationName='Ясная Поляна');
""")
cur.fetchall()

[{'originalLocation': 'Ясная Поляна.'},
 {'originalLocation': 'Ясная Поляна'},
 {'originalLocation': 'деревня Ясная Поляна'},
 {'originalLocation': 'Усадьба Ясная Поляна'},
 {'originalLocation': 'Деревня Ясная Поляна.'},
 {'originalLocation': 'в Ясная Поляна'},
 {'originalLocation': 'Ясеая Поляна'},
 {'originalLocation': 'Ясная \nПоляна.'},
 {'originalLocation': 'ЯснаяПоляна'},
 {'originalLocation': 'Ясная Поляна (?).'},
 {'originalLocation': 'Ясная Полана'},
 {'originalLocation': 'Овсянниково'},
 {'originalLocation': 'Ясная Поляна (?)'},
 {'originalLocation': 'Усадьба Ясная Поляна.'},
 {'originalLocation': 'Ясная Поляна ?'},
 {'originalLocation': 'Ясная Поляна?'},
 {'originalLocation': 'Ясная Поляна. (?)'},
 {'originalLocation': 'Ясная Поляна .(?)'}]

### `SELECT` с `JOIN` и большим количеством условий

Найдём все такие фотографии, которые были сделаны до смерти ЛНТ (1910), где что-нибудь написано на обороте (обычно это какие-то пожелания или подписи людей на фото) и которым приписана хоть одна рубрика, а ещё выведем пути к их полным версиям.

In [13]:
cur.execute("""
SELECT 
	COUNT(PD.idPhoto)
FROM PhotoDescriptions AS PD
	JOIN MuseumAnnotations AS MA ON PD.idPhoto=MA.idPhoto
	JOIN PhotoPhysicalDescriptions AS PPD on PD.idPhoto=PPD.idPhoto
	JOIN Locations AS L ON L.idLocation=PD.idLocation
	JOIN PhotoFiles AS PF ON PD.idPhoto=PF.idPhoto
WHERE
	PPD.inscription <> '-1'
    AND PD.year < 1910
    AND L.locationName <> 'Ясная Поляна'
""")
cur.fetchall()

[{'COUNT(PD.idPhoto)': 36}]

А вот одна из них — командная (id 67473).

![](./data/fullsize/00067473.jpg)

In [9]:
cur.close()

True